# **mnist_convnet**
- ELEC 576 HW 1
- Robert Heeter
- 4 October 2023

## **Structure**:
1) Set PyTorch metadata
    - Seed
    - TensorFlow output
    - Whether to transfer to gpu (cuda)
2) Import data
    - Download data
    - Create data loaders with batchsize, transforms, scaling
3) Define model architecture, loss, and optimizer
4) Define test and training loops
    - Train:
        - Get next batch
        - Forward pass through model-
        - Calculate loss
        - Backward pass from loss (calculates the gradient for each parameter)
        - Optimizer: performs weight updates
5) Perform training over multiple epochs
    - Each epoch:
        - Call train loop
        - Call test loop

## **Acknowledgements**:
- https://colab.research.google.com/drive/1i9KpbQyFU4zfq8zLLns8a2Kd8PRMGsaZ
- https://github.com/motokimura/pytorch_tensorboard/blob/master/main.py
- https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb#scrollTo=lpUO9HqUKP6z
- https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-tensorboard-with-pytorch.md


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os

%load_ext tensorboard


2023-10-06 01:46:54.245715: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# 1. Set PyTorch metadata

batch_size = 64
test_batch_size = 1000
epochs = 10
lr = 0.01
try_cuda = True
seed = 1000
logging_interval = 10 # how many batches to wait before logging
logging_dir = None

# setting up the logging
log_dir = os.path.join(os.getcwd(),'log/MNIST', datetime.now().strftime('%b%d_%H-%M-%S'))
writer = SummaryWriter(log_dir=log_dir)

# deciding whether to send to the cpu or not if available
if torch.cuda.is_available() and try_cuda:
    cuda = True
    torch.cuda.mnaual_seed(seed)
else:
    cuda = False
    torch.manual_seed(seed)
    

In [3]:
# 2. Import data

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.01307,), (0.3081,))])

train_loader = torch.utils.data.DataLoader(datasets.MNIST('data', train=True, download=True, transform=transform),
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.MNIST('data', train=False, download=True, transform=transform),
                                          batch_size=test_batch_size,
                                          shuffle=True)


In [4]:
# 3. Defining model architecture, loss, and optimizer

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.layers = nn.Sequential(nn.Conv2d(1, 10, kernel_size=5),
                                    nn.Tanh(),
                                    nn.MaxPool2d(2),
                                    nn.Conv2d(10, 20, kernel_size=5),
                                    nn.Tanh(),
                                    nn.MaxPool2d(2),
                                    nn.Flatten(),
                                    nn.Linear(320, 50),
                                    nn.Tanh(),
                                    nn.Dropout2d(0.5),
                                    nn.Linear(50, 10),
                                    nn.Softmax(dim=1)
                                   )
        
    def forward(self, x):
        '''Forward pass'''
        return self.layers(x)


#         self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
#         self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
#         self.conv2_drop = nn.Dropout2d()
#         self.fc1 = nn.Linear(320, 50)
#         self.fc2 = nn.Linear(50, 10)

#     def forward(self, x):
#         # original network architecture
#         # x = F.relu(F.max_pool2d(self.conv1(x), 2))
#         # x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
#         # x = x.view(-1, 320) # (batch_size, units)
#         # x = F.relu(self.fc1(x))
#         # x = F.dropout(x, training=self.training)
#         # x = self.fc2(x)
#         # x = F.softmax(x, dim=1)

#         # new network architecture
#         x = self.conv1(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, 2)
#         x = self.conv2(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, 2)
#         x = x.view(-1, 320)
#         x = self.fc1(x)
#         x = F.relu(x)
#         x = F.dropout(x, p=0.5)
#         x = self.fc2(x)
#         x = F.softmax(x, dim=1)

#         return x

model = Net()
print(model)

optimizer = optim.SGD(model.parameters(), lr=lr)
print(optimizer)


Net(
  (layers): Sequential(
    (0): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=320, out_features=50, bias=True)
    (8): Tanh()
    (9): Dropout2d(p=0.5, inplace=False)
    (10): Linear(in_features=50, out_features=10, bias=True)
    (11): Softmax(dim=1)
  )
)
SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.01
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [5]:
def weight_histograms_conv2d(writer, n_iter, weights, biases, layer_number):
    weights_shape = weights.shape
    num_kernels = weights_shape[0]
    
    for k in range(num_kernels):
        
        flattened_weights = weights[k].flatten()
        tag = f"layer_{layer_number}/kernel_{k}_weight_histogram"
        writer.add_histogram(tag, flattened_weights, n_iter, bins='tensorflow')
        
        tag = f"layer_{layer_number}/kernel_{k}_weight_min"
        writer.add_scalar(tag, torch.min(flattened_weights), n_iter)
        tag = f"layer_{layer_number}/kernel_{k}_weight_max"
        writer.add_scalar(tag, torch.max(flattened_weights), n_iter)
        tag = f"layer_{layer_number}/kernel_{k}_weight_mean"
        writer.add_scalar(tag, torch.mean(flattened_weights), n_iter)
        tag = f"layer_{layer_number}/kernel_{k}_weight_stdev"
        writer.add_scalar(tag, torch.std(flattened_weights), n_iter)
        
        flattened_biases = biases[k].flatten()
        tag = f"layer_{layer_number}/kernel_{k}_biases_histogram"
        writer.add_histogram(tag, flattened_biases, n_iter, bins='tensorflow')
        
        tag = f"layer_{layer_number}/kernel_{k}_biases_min"
        writer.add_scalar(tag, torch.min(flattened_biases), n_iter)
        tag = f"layer_{layer_number}/kernel_{k}_biases_max"
        writer.add_scalar(tag, torch.max(flattened_biases), n_iter)
        tag = f"layer_{layer_number}/kernel_{k}_biases_mean"
        writer.add_scalar(tag, torch.mean(flattened_biases), n_iter)
        tag = f"layer_{layer_number}/kernel_{k}_biases_stdev"
        writer.add_scalar(tag, torch.std(flattened_biases), n_iter)
        
        break
        
def weight_histograms_linear(writer, n_iter, weights, biases, layer_number):
    flattened_weights = weights.flatten()
    tag = f"layer_{layer_number}_weight_histogram"
    writer.add_histogram(tag, flattened_weights, n_iter, bins='tensorflow')
    
    tag = f"layer_{layer_number}_weight_min"
    writer.add_scalar(tag, torch.min(flattened_weights), n_iter)
    tag = f"layer_{layer_number}_weight_max"
    writer.add_scalar(tag, torch.max(flattened_weights), n_iter)
    tag = f"layer_{layer_number}_weight_mean"
    writer.add_scalar(tag, torch.mean(flattened_weights), n_iter)
    tag = f"layer_{layer_number}_weight_stdev"
    writer.add_scalar(tag, torch.std(flattened_weights), n_iter)

    flattened_biases = biases.flatten()
    tag = f"layer_{layer_number}_biases_histogram"
    writer.add_histogram(tag, flattened_biases, n_iter, bins='tensorflow')

    tag = f"layer_{layer_number}_biases_min"
    writer.add_scalar(tag, torch.min(flattened_biases), n_iter)
    tag = f"layer_{layer_number}_biases_max"
    writer.add_scalar(tag, torch.max(flattened_biases), n_iter)
    tag = f"layer_{layer_number}_biases_mean"
    writer.add_scalar(tag, torch.mean(flattened_biases), n_iter)
    tag = f"layer_{layer_number}_biases_stdev"
    writer.add_scalar(tag, torch.std(flattened_biases), n_iter)

def weight_histograms(writer, n_iter, model):
    # iterate over all model layers
    for layer_number in range(len(model.layers)):
        
        # get layer
        layer = model.layers[layer_number]
        
        # compute weight histograms for appropriate layer
        if isinstance(layer, nn.Conv2d):
            weights = layer.weight
            biases = layer.bias
            weight_histograms_conv2d(writer, n_iter, weights, biases, layer_number)
            
        elif isinstance(layer, nn.Linear):
            weights = layer.weight
            biases = layer.bias
            weight_histograms_linear(writer, n_iter, weights, biases, layer_number)


In [7]:
# 4. Define test and training loops

eps = 1e-13

def train(epoch):
    model.train()

    # criterion = nn.CrossEntropyLoss()
    criterion = nn.NLLLoss(size_average=False)
    
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()

        optimizer.zero_grad()
        output = model(data) # forward pass
        loss = criterion(torch.log(output+eps), target) # = sum_k(-t_k * log(y_k))
        loss.backward() # backward pass
        optimizer.step()

        if batch_idx % logging_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data)
            )

            # log train/loss to TensorBoard at every iteration
            n_iter = (epoch - 1) * len(train_loader) + batch_idx + 1
            writer.add_scalar('train/loss', loss.data, n_iter)
                    
    # visualize layer weights and biases
    weight_histograms(writer, n_iter, model)
    
    # log model parameters to TensorBoard at every epoch
    for name, param in model.named_parameters():
        layer, attr = os.path.splitext(name)
        attr = attr[1:]
        writer.add_histogram('{}/{}'.format(layer, attr), param.clone().cpu().data.numpy(), n_iter)

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    
    # criterion = nn.CrossEntropyLoss()
    # criterion = nn.CrossEntropyLoss(size_average = False)
    criterion = nn.NLLLoss(size_average = False)

    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()

        output = model(data)

        test_loss += criterion(torch.log(output+eps), target,).item() # sum up batch loss (later, averaged over all test samples)
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), test_accuracy)
    )

    # log test/loss and test/accuracy to TensorBoard at every epoch
    n_iter = epoch * len(train_loader)
    writer.add_scalar('test/loss', test_loss, n_iter)
    writer.add_scalar('test/accuracy', test_accuracy, n_iter)
    

In [8]:
# 5. Perform training over multiple epochs

# start training
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

writer.close()


/Users/rch/opt/anaconda3/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/Users/rch/opt/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Train Epoch: 1 [0/60000 (0%)]	Loss: 148.350174
Train Epoch: 1 [640/60000 (1%)]	Loss: 65.574791
Train Epoch: 1 [1280/60000 (2%)]	Loss: 50.805614
Train Epoch: 1 [1920/60000 (3%)]	Loss: 27.808430
Train Epoch: 1 [2560/60000 (4%)]	Loss: 29.381617
Train Epoch: 1 [3200/60000 (5%)]	Loss: 47.149063
Train Epoch: 1 [3840/60000 (6%)]	Loss: 27.568836
Train Epoch: 1 [4480/60000 (7%)]	Loss: 25.695684
Train Epoch: 1 [5120/60000 (9%)]	Loss: 32.483009
Train Epoch: 1 [5760/60000 (10%)]	Loss: 28.119480
Train Epoch: 1 [6400/60000 (11%)]	Loss: 16.325533
Train Epoch: 1 [7040/60000 (12%)]	Loss: 21.478676
Train Epoch: 1 [7680/60000 (13%)]	Loss: 23.415968
Train Epoch: 1 [8320/60000 (14%)]	Loss: 21.872169
Train Epoch: 1 [8960/60000 (15%)]	Loss: 13.121871
Train Epoch: 1 [9600/60000 (16%)]	Loss: 13.346699
Train Epoch: 1 [10240/60000 (17%)]	Loss: 30.787872
Train Epoch: 1 [10880/60000 (18%)]	Loss: 22.634539
Train Epoch: 1 [11520/60000 (19%)]	Loss: 25.395401
Train Epoch: 1 [12160/60000 (20%)]	Loss: 14.447589
Train Ep

Train Epoch: 2 [44800/60000 (75%)]	Loss: 7.591688
Train Epoch: 2 [45440/60000 (76%)]	Loss: 16.178593
Train Epoch: 2 [46080/60000 (77%)]	Loss: 6.151915
Train Epoch: 2 [46720/60000 (78%)]	Loss: 16.359280
Train Epoch: 2 [47360/60000 (79%)]	Loss: 16.866432
Train Epoch: 2 [48000/60000 (80%)]	Loss: 5.174510
Train Epoch: 2 [48640/60000 (81%)]	Loss: 7.196651
Train Epoch: 2 [49280/60000 (82%)]	Loss: 6.357285
Train Epoch: 2 [49920/60000 (83%)]	Loss: 16.031305
Train Epoch: 2 [50560/60000 (84%)]	Loss: 5.842550
Train Epoch: 2 [51200/60000 (85%)]	Loss: 20.573612
Train Epoch: 2 [51840/60000 (86%)]	Loss: 5.943273
Train Epoch: 2 [52480/60000 (87%)]	Loss: 2.580393
Train Epoch: 2 [53120/60000 (88%)]	Loss: 7.100457
Train Epoch: 2 [53760/60000 (90%)]	Loss: 26.377951
Train Epoch: 2 [54400/60000 (91%)]	Loss: 7.567432
Train Epoch: 2 [55040/60000 (92%)]	Loss: 3.035346
Train Epoch: 2 [55680/60000 (93%)]	Loss: 2.375730
Train Epoch: 2 [56320/60000 (94%)]	Loss: 12.264088
Train Epoch: 2 [56960/60000 (95%)]	Loss: 8.

Train Epoch: 4 [28800/60000 (48%)]	Loss: 11.116791
Train Epoch: 4 [29440/60000 (49%)]	Loss: 2.402918
Train Epoch: 4 [30080/60000 (50%)]	Loss: 4.401239
Train Epoch: 4 [30720/60000 (51%)]	Loss: 4.655196
Train Epoch: 4 [31360/60000 (52%)]	Loss: 4.891407
Train Epoch: 4 [32000/60000 (53%)]	Loss: 2.961356
Train Epoch: 4 [32640/60000 (54%)]	Loss: 10.901689
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.554322
Train Epoch: 4 [33920/60000 (57%)]	Loss: 9.157825
Train Epoch: 4 [34560/60000 (58%)]	Loss: 6.211045
Train Epoch: 4 [35200/60000 (59%)]	Loss: 3.544980
Train Epoch: 4 [35840/60000 (60%)]	Loss: 5.057472
Train Epoch: 4 [36480/60000 (61%)]	Loss: 3.552627
Train Epoch: 4 [37120/60000 (62%)]	Loss: 2.418842
Train Epoch: 4 [37760/60000 (63%)]	Loss: 12.234315
Train Epoch: 4 [38400/60000 (64%)]	Loss: 2.502735
Train Epoch: 4 [39040/60000 (65%)]	Loss: 5.953590
Train Epoch: 4 [39680/60000 (66%)]	Loss: 8.780099
Train Epoch: 4 [40320/60000 (67%)]	Loss: 1.174813
Train Epoch: 4 [40960/60000 (68%)]	Loss: 4.3576

Train Epoch: 6 [12160/60000 (20%)]	Loss: 10.302482
Train Epoch: 6 [12800/60000 (21%)]	Loss: 6.445203
Train Epoch: 6 [13440/60000 (22%)]	Loss: 1.047648
Train Epoch: 6 [14080/60000 (23%)]	Loss: 13.805943
Train Epoch: 6 [14720/60000 (25%)]	Loss: 6.265696
Train Epoch: 6 [15360/60000 (26%)]	Loss: 4.532309
Train Epoch: 6 [16000/60000 (27%)]	Loss: 9.401494
Train Epoch: 6 [16640/60000 (28%)]	Loss: 3.560211
Train Epoch: 6 [17280/60000 (29%)]	Loss: 6.512724
Train Epoch: 6 [17920/60000 (30%)]	Loss: 6.915971
Train Epoch: 6 [18560/60000 (31%)]	Loss: 6.477233
Train Epoch: 6 [19200/60000 (32%)]	Loss: 1.410539
Train Epoch: 6 [19840/60000 (33%)]	Loss: 13.338208
Train Epoch: 6 [20480/60000 (34%)]	Loss: 3.021915
Train Epoch: 6 [21120/60000 (35%)]	Loss: 13.437016
Train Epoch: 6 [21760/60000 (36%)]	Loss: 2.353795
Train Epoch: 6 [22400/60000 (37%)]	Loss: 1.897857
Train Epoch: 6 [23040/60000 (38%)]	Loss: 7.407558
Train Epoch: 6 [23680/60000 (39%)]	Loss: 9.312606
Train Epoch: 6 [24320/60000 (41%)]	Loss: 7.692

Train Epoch: 7 [56960/60000 (95%)]	Loss: 8.735505
Train Epoch: 7 [57600/60000 (96%)]	Loss: 2.060519
Train Epoch: 7 [58240/60000 (97%)]	Loss: 4.416695
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.822052
Train Epoch: 7 [59520/60000 (99%)]	Loss: 1.440630

Test set: Average loss: 0.0552, Accuracy: 9831/10000 (98.31%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 3.563952
Train Epoch: 8 [640/60000 (1%)]	Loss: 5.155003
Train Epoch: 8 [1280/60000 (2%)]	Loss: 2.238763
Train Epoch: 8 [1920/60000 (3%)]	Loss: 7.062019
Train Epoch: 8 [2560/60000 (4%)]	Loss: 3.042124
Train Epoch: 8 [3200/60000 (5%)]	Loss: 2.825757
Train Epoch: 8 [3840/60000 (6%)]	Loss: 6.655331
Train Epoch: 8 [4480/60000 (7%)]	Loss: 9.738765
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.335297
Train Epoch: 8 [5760/60000 (10%)]	Loss: 1.882492
Train Epoch: 8 [6400/60000 (11%)]	Loss: 1.940750
Train Epoch: 8 [7040/60000 (12%)]	Loss: 4.579886
Train Epoch: 8 [7680/60000 (13%)]	Loss: 11.259303
Train Epoch: 8 [8320/60000 (14%)]	Loss: 2.946391
Train Epoch:

Train Epoch: 9 [41600/60000 (69%)]	Loss: 6.284098
Train Epoch: 9 [42240/60000 (70%)]	Loss: 7.998821
Train Epoch: 9 [42880/60000 (71%)]	Loss: 14.846068
Train Epoch: 9 [43520/60000 (72%)]	Loss: 5.223794
Train Epoch: 9 [44160/60000 (74%)]	Loss: 8.336000
Train Epoch: 9 [44800/60000 (75%)]	Loss: 3.820257
Train Epoch: 9 [45440/60000 (76%)]	Loss: 12.239676
Train Epoch: 9 [46080/60000 (77%)]	Loss: 8.983435
Train Epoch: 9 [46720/60000 (78%)]	Loss: 1.897945
Train Epoch: 9 [47360/60000 (79%)]	Loss: 1.106027
Train Epoch: 9 [48000/60000 (80%)]	Loss: 11.650100
Train Epoch: 9 [48640/60000 (81%)]	Loss: 2.005645
Train Epoch: 9 [49280/60000 (82%)]	Loss: 2.290253
Train Epoch: 9 [49920/60000 (83%)]	Loss: 3.807944
Train Epoch: 9 [50560/60000 (84%)]	Loss: 9.133227
Train Epoch: 9 [51200/60000 (85%)]	Loss: 2.091491
Train Epoch: 9 [51840/60000 (86%)]	Loss: 1.488215
Train Epoch: 9 [52480/60000 (87%)]	Loss: 10.162395
Train Epoch: 9 [53120/60000 (88%)]	Loss: 1.763453
Train Epoch: 9 [53760/60000 (90%)]	Loss: 7.418

In [9]:
%tensorboard --logdir log/MNIST --port=8008
